---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kpcrotty.github.io/)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
# start and end years

start_yr = 1980
stop_yr  = 2022


# GET DATA

In [2]:
import pandas as pd
import yfinance as yf
from pandas_datareader import DataReader as pdr
import statsmodels.api as sm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

start = str(start_yr) + "-01"
stop  = str(stop_yr) + "-12"

# get monthly market excess return, risk-free rate, and value-weighted industry returns
ff = pdr('F-F_Research_Data_Factors','famafrench',start=1970)[0] 
df = pdr('48_Industry_Portfolios','famafrench',start=1970)[0] 
inds = df.columns

# merge with market excess return and risk-free rate
df = df.join(ff,how='inner')

# compute excess industry returns
df[inds] = df[inds].subtract(df.RF,axis='index')

# CALCULATIONS

In [3]:
# compute anualized industry mean excess returns
d1 = df.loc[start:stop]
d2 = pd.DataFrame(dtype=float,index=inds,columns=['rprem','beta'])
d2['rprem'] = 12*d1[inds].mean()

# run regressions to compute industry betas
X = sm.add_constant(d1["Mkt-RF"])
def beta(y):
    return sm.OLS(y, X).fit().params["Mkt-RF"]
betas = d1[inds].apply(beta)

# calculate empirical and theoretical risk premia
d2 = pd.DataFrame(
    dtype=float,
    index=inds,
    columns=["beta", "empirical rprem", "theoretical rprem"],
)
d2.index.name = "industry"
d2["beta"] = betas
d2["empirical rprem"] = 12 * d1[inds].mean() / 100 
d2["theoretical rprem"] = 12 * betas * d1["Mkt-RF"].mean() / 100 
d2 = d2.reset_index()

# FIGURE

In [4]:
import plotly.express as px
import plotly.graph_objects as go

fig1 = px.scatter(
    d2, x="beta", y="empirical rprem", hover_name="industry", trendline="ols",
)
fig1.update_traces(
    marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)

fig2 = px.scatter(d2, x="beta", y="theoretical rprem", hover_name="industry")
fig2.update_traces(
    marker=dict(size=12, color='green', line=dict(width=2, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)
fig = go.Figure(data=fig1.data + fig2.data)

fig.layout.xaxis["title"] = "Beta"
fig.layout.yaxis["title"] = "Risk Premium (Annualized)"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".2")
fig.show()